In [2]:
import os
import sys
project_dir = os.path.join(os.pardir, os.pardir)
sys.path.append(project_dir)

import dotenv
dotenv_path = os.path.join(project_dir, '.env')
dotenv.load_dotenv(dotenv_path)

from IPython.display import display

import glob
import time
import numpy as np
import pandas as pd

from src.data.processing_func import (connect_database, extract_geo_sections)

Get processed datasets

In [3]:
filenames = glob.glob(project_dir+"/data/interim/temp_df_jps_2017_10_1*")
filenames.sort()
frame = pd.DataFrame()
list_ = []
for file_ in filenames:
    df = pd.read_csv(file_, index_col=0)
    list_.append(df)
df_jps = pd.concat(list_)
display(df_jps.shape)
display(df_jps.sample(5))

(760635, 18)

,MgrcDateStart,JpsId,SctnId,JamId,JamIndLevelOfTraffic,JamQtdLengthMeters,JamSpdMetersPerSecond,JamTimeDelayInSeconds,JamDscCoordinatesLonLat,JamSpdKmPerHour,LonDirection,LatDirection,MajorDirection,date,hour,minute,period,minute_bin
11169,2017-12-04 18:37:00-02:00,7540624.0,10633.0,741265.0,3.0,1960.0,2.736111,526.0,"[{'x': -48.847909, 'y': -26.275415}, {'x': -48...",9.85,West,North,Leste/Oeste,2017-12-04,18,37,1,30 a 44
32778,2017-12-20 18:57:00-02:00,7934519.0,4824.0,832011.0,4.0,1660.0,1.891667,682.0,"[{'x': -48.841069, 'y': -26.305699}, {'x': -48...",6.81,East,South,Norte/Sul,2017-12-20,18,57,1,45 a 59
20451,2017-11-14 17:32:00-02:00,7129916.0,15395.0,653664.0,3.0,498.0,2.272222,162.0,"[{'x': -48.840585, 'y': -26.316179}, {'x': -48...",8.18,East,South,Norte/Sul,2017-11-14,17,32,1,30 a 44
37274,2017-12-14 17:37:00-02:00,7794874.0,8076.0,799060.0,3.0,454.0,2.666667,113.0,"[{'x': -48.852581, 'y': -26.293511}, {'x': -48...",9.60,East,South,Leste/Oeste,2017-12-14,17,37,1,30 a 44
50822,2017-11-23 18:07:00-02:00,7319115.0,14606.0,693523.0,4.0,456.0,1.777778,221.0,"[{'x': -48.862578, 'y': -26.303707}, {'x': -48...",6.40,West,North,Norte/Sul,2017-11-23,18,7,1,0 a 14


Get data from SIG and merge with Waze's data:

In [4]:
#Connection and initial setup
DATABASE = {
'drivername': os.environ.get("db_drivername"),
'host': os.environ.get("db_host"), 
'port': os.environ.get("db_port"),
'username': os.environ.get("db_username"),
'password': os.environ.get("db_password"),
'database': os.environ.get("db_database"),
}

meta = connect_database(DATABASE)
geo_sections = extract_geo_sections(meta, main_buffer=10)
df_waze_sig = geo_sections.merge(df_jps, how="inner", on="SctnId")
display(df_waze_sig.shape)
display(df_waze_sig.head())

(760468, 39)

,SctnId,SctnIdArcgis,SctnCodRua,SctnDscNome,SctnQtdMetrosAcumulados,SctnQtdComprimento,SctnDscCoordxUtmComeco,SctnDscCoordyUtmComeco,SctnDscCoordxUtmMeio,SctnDscCoordyUtmMeio,...,JamDscCoordinatesLonLat,JamSpdKmPerHour,LonDirection,LatDirection,MajorDirection,date,hour,minute,period,minute_bin
0,0,16092,2686,DR. JOAO COLIN,2335,89.087809,714734.1111,7091467.602,714737.35335,7.091423e+06,...,"[{'x': -48.849316, 'y': -26.282206}, {'x': -48...",17.32,West,North,Norte/Sul,2017-10-02,18,31,1,30 a 44
1,0,16092,2686,DR. JOAO COLIN,2335,89.087809,714734.1111,7091467.602,714737.35335,7.091423e+06,...,"[{'y': -26.281302, 'x': -48.8494}, {'y': -26.2...",12.65,West,North,Norte/Sul,2017-10-04,18,31,1,30 a 44
2,0,16092,2686,DR. JOAO COLIN,2335,89.087809,714734.1111,7091467.602,714737.35335,7.091423e+06,...,"[{'y': -26.281302, 'x': -48.8494}, {'y': -26.2...",12.70,West,North,Norte/Sul,2017-10-04,18,36,1,30 a 44
3,0,16092,2686,DR. JOAO COLIN,2335,89.087809,714734.1111,7091467.602,714737.35335,7.091423e+06,...,"[{'x': -48.8494, 'y': -26.281302}, {'x': -48.8...",16.86,West,North,Norte/Sul,2017-12-07,18,17,1,15 a 29
4,0,16092,2686,DR. JOAO COLIN,2335,89.087809,714734.1111,7091467.602,714737.35335,7.091423e+06,...,"[{'x': -48.849316, 'y': -26.282206}, {'x': -48...",10.53,West,North,Norte/Sul,2017-12-07,18,32,1,30 a 44


Get undirected Traffic Network

In [5]:
un_network = pd.read_csv(project_dir + "/data/external/od/Tabela_atributos_CSV.csv",
                         encoding="latin-1",
                         sep=";",
                         decimal=',')
un_network.head()
col_dict={'objectid,N,19,11': 'SctnIdArcgis',
            'codlogra,N,19,11': 'SctnCodRua',
            'acumulo,N,19,11': 'SctnQtdMetrosAcumulados',
            'nomelog,C,254': 'SctnDscNome',
            'x_ini,N,19,11': 'x_ini',
            'y_ini,N,19,11': 'y_ini',
            'x_med,N,19,11': 'x_med',
            'y_med,N,19,11': 'y_med',
            'x_fim,N,19,11': 'x_fim',
            'y_fim,N,19,11': 'y_fim',
         }
un_network.rename(columns=col_dict, inplace=True)
un_network = un_network[list(col_dict.values())]
un_network.drop_duplicates(["SctnCodRua", "SctnQtdMetrosAcumulados"], inplace=True)
un_network.set_index(["SctnCodRua", "SctnQtdMetrosAcumulados"], inplace=True, verify_integrity=True)
un_network = un_network.astype(int, errors='ignore')
un_network["initial_node"] = list(zip(un_network.x_ini, un_network.y_ini))
un_network["final_node"] = list(zip(un_network.x_fim, un_network.y_fim))
un_network.sample(5)

,,SctnIdArcgis,SctnDscNome,x_ini,y_ini,x_med,y_med,x_fim,y_fim,initial_node,final_node
SctnCodRua,SctnQtdMetrosAcumulados,,,,,,,,,,
7186.0,1326.0,27063,PAPA JOAO XXIII,717817,7091408,717787,7091444,717757,7091480,"(717817, 7091408)","(717757, 7091480)"
1230.0,1264.0,17436,BENJAMIN CONSTANT,713514,7090964,713528,7090963,713543,7090962,"(713514, 7090964)","(713543, 7090962)"
4897.0,1968.0,25521,IRIRIU,716978,7092436,716995,7092447,717012,7092459,"(716978, 7092436)","(717012, 7092459)"
130.0,1089.0,16959,GETULIO VARGAS,715091,7087478,715092,7087437,715094,7087395,"(715091, 7087478)","(715094, 7087395)"
9253.0,481.0,25738,TENENTE PAULO LOPES,718419,7091303,718398,7091332,718377,7091360,"(718419, 7091303)","(718377, 7091360)"


Get number of directions per section

In [6]:
def get_two_ways(row):
    if row["SectionDirection"] == "Norte/Sul":
        if row["LatDirection"] == 2:
            return "two_way"
    
    if row["SectionDirection"] == "Leste/Oeste":
        if row["LonDirection"] == 2:
            return "two_way"
        
    return "unknown"

df_waze_directions = df_waze_sig.groupby(["SctnCodRua",
                                          "SctnQtdMetrosAcumulados",
                                          "SctnDscNome",
                                          "SectionDirection",
                                          "SctnDscCoordxUtmComeco",
                                          "SctnDscCoordyUtmComeco",
                                          "SctnDscCoordxUtmFinal",
                                          "SctnDscCoordyUtmFinal"]).agg({"LatDirection": pd.Series.nunique,
                                                                           "LonDirection": pd.Series.nunique
                                                                          })
df_waze_directions["total_directions"] = df_waze_directions["LatDirection"] + df_waze_directions["LonDirection"]
df_waze_directions.reset_index(inplace=True)
df_waze_directions["way"] = df_waze_directions.apply(get_two_ways, axis=1)
df_waze_directions.drop_duplicates(subset=["SctnCodRua", "SctnQtdMetrosAcumulados"], inplace=True)
df_waze_directions.set_index(["SctnCodRua", "SctnQtdMetrosAcumulados"], inplace=True, verify_integrity=True)
df_waze_directions.sort_values("total_directions", ascending=False, inplace=True)
df_waze_directions.head()

SctnDscNome SectionDirection  \
SctnCodRua SctnQtdMetrosAcumulados                                          
2691       380                      DOUTOR PLACIDO GOMES      Leste/Oeste   
8274       4410                       QUINZE DE NOVEMBRO        Norte/Sul   
3497       324                                   ESPIGAO        Norte/Sul   
           400                                   ESPIGAO        Norte/Sul   
3505       2149                                 BARBANTE        Norte/Sul   

                                    SctnDscCoordxUtmComeco  \
SctnCodRua SctnQtdMetrosAcumulados                           
2691       380                                 714960.2945   
8274       4410                                711545.2532   
3497       324                                 720725.6297   
           400                                 720690.8223   
3505       2149                                708208.3280   

                                    SctnDscCoordyUtmComeco  \
SctnCodRua SctnQtdMetrosAcumulados                           
2691       380                                7.088144e+06   
8274       4410                               7.089831e+06   
3497       324                                7.084441e+06   
           400                                7.084508e+06   
3505       2149                               7.087394e+06   

                                    SctnDscCoordxUtmFinal  \
SctnCodRua SctnQtdMetrosAcumulados                          
2691       380                                715074.0814   
8274       4410                               711601.0300   
3497       324                                720740.2162   
           400                                720725.6297   
3505       2149                               707425.4000   

                                    SctnDscCoordyUtmFinal  LatDirection  \
SctnCodRua SctnQtdMetrosAcumulados                                        
2691       380                               7.088149e+06             2   
8274       4410                              7.089977e+06             2   
3497       324                               7.084416e+06             2   
           400                               7.084441e+06             2   
3505       2149                              7.088275e+06             2   

                                    LonDirection  total_directions      way  
SctnCodRua SctnQtdMetrosAcumulados                                           
2691       380                                 2                 4  two_way  
8274       4410                                2                 4  two_way  
3497       324                                 2                 4  two_way  
           400                                 2                 4  two_way  
3505       2149                                2                 4  two_way

Cross-checking classification:

In [70]:
directions_gb = df_waze_directions.groupby(["SctnDscNome", "way"]).agg({"way": "count"}).sort_values("way",
                                                                                     ascending=False)
directions_gb.xs("two_way", level="way", drop_level=False)
#directions_gb.to_csv(project_dir + "/data/interim/od/check_sentidos.csv")
df_waze_directions[df_waze_directions.SctnDscNome == "VISCONDE DE TAUNAY"]
df_waze_sig[(df_waze_sig.SctnCodRua==9719) & (df_waze_sig.SctnQtdMetrosAcumulados==498)][["SctnDscNome",
                                                                                          "SctnId",
                                                                                          "SctnDscCoordxUtmComeco",
                                                                                          "SctnDscCoordyUtmComeco",
                                                                                          "SctnDscCoordxUtmFinal",
                                                                                          "SctnDscCoordyUtmFinal",
                                                                                          "MgrcDateStart",
                                                                                          "JamId",
                                                                                          "LonDirection",
                                                                                          "LatDirection",
                                                                                          "StreetDirection",
                                                                                          "SectionDirection",
                                                                                          "MajorDirection",
                                                                                          "JamQtdLengthMeters"
                                                                                         ]]

,SctnDscNome,SctnId,SctnDscCoordxUtmComeco,SctnDscCoordyUtmComeco,SctnDscCoordxUtmFinal,SctnDscCoordyUtmFinal,MgrcDateStart,JamId,LonDirection,LatDirection,StreetDirection,SectionDirection,MajorDirection,JamQtdLengthMeters
331117,VISCONDE DE TAUNAY,7434,714603.3836,7088915.131,714469.7817,7.088789e+06,2017-10-02 18:21:00-03:00,479047.0,West,South,Norte/Sul,Leste/Oeste,Leste/Oeste,348.0
331118,VISCONDE DE TAUNAY,7434,714603.3836,7088915.131,714469.7817,7.088789e+06,2017-10-02 18:31:00-03:00,479150.0,West,South,Norte/Sul,Leste/Oeste,Norte/Sul,1127.0
331119,VISCONDE DE TAUNAY,7434,714603.3836,7088915.131,714469.7817,7.088789e+06,2017-10-03 18:26:00-03:00,481589.0,West,South,Norte/Sul,Leste/Oeste,Leste/Oeste,348.0
331120,VISCONDE DE TAUNAY,7434,714603.3836,7088915.131,714469.7817,7.088789e+06,2017-12-04 18:27:00-02:00,740930.0,West,South,Norte/Sul,Leste/Oeste,Norte/Sul,1129.0
331121,VISCONDE DE TAUNAY,7434,714603.3836,7088915.131,714469.7817,7.088789e+06,2017-12-04 18:27:00-02:00,740931.0,East,North,Norte/Sul,Leste/Oeste,Leste/Oeste,161.0
331122,VISCONDE DE TAUNAY,7434,714603.3836,7088915.131,714469.7817,7.088789e+06,2017-12-04 18:32:00-02:00,741067.0,West,South,Norte/Sul,Leste/Oeste,Norte/Sul,1110.0
331123,VISCONDE DE TAUNAY,7434,714603.3836,7088915.131,714469.7817,7.088789e+06,2017-12-04 18:32:00-02:00,741068.0,East,North,Norte/Sul,Leste/Oeste,Leste/Oeste,161.0
331124,VISCONDE DE TAUNAY,7434,714603.3836,7088915.131,714469.7817,7.088789e+06,2017-12-04 18:37:00-02:00,741199.0,West,South,Norte/Sul,Leste/Oeste,Norte/Sul,1129.0
331125,VISCONDE DE TAUNAY,7434,714603.3836,7088915.131,714469.7817,7.088789e+06,2017-12-04 18:42:00-02:00,741321.0,West,South,Norte/Sul,Leste/Oeste,Norte/Sul,999.0
331126,VISCONDE DE TAUNAY,7434,714603.3836,7088915.131,714469.7817,7.088789e+06,2017-12-05 18:02:00-02:00,746501.0,West,North,Norte/Sul,Leste/Oeste,Leste/Oeste,192.0


In [72]:
df_jps[df_jps.JamId == 740931]["JamDscCoordinatesLonLat"].iloc[0]

"[{'x': -48.850465, 'y': -26.303469}, {'x': -48.850301, 'y': -26.303503}, {'x': -48.84983, 'y': -26.303093}, {'x': -48.849674, 'y': -26.302957}, {'x': -48.849264, 'y': -26.3026}]"

Generate traffic network duplicating two-way streets. To achieve that we will assume that any street that contains AT LEAST ONE two-way section is two-way in its entirety.

In [58]:
un_network_join = un_network.join(df_waze_directions["way"])
un_network_join = un_network_join.groupby("SctnDscNome").agg({"way": lambda x: "two_way" in x.values})
un_network_join[un_network_join.way == True]


,way
SctnDscNome,
6 DE JANEIRO,True
ADOLFO DA VEIGA,True
AGULHAS NEGRAS,True
ALBANO SCHMIDT,True
ALMIRANTE JACEGUAY,True
ALOIS FINDER,True
AMERICO VESPUCIO,True
ANABURGO,True
ANITA GARIBALDI,True
